In [ ]:
import CoolProp.CoolProp as CP
import cantera as ct
import math
from rocketcea.cea_obj import CEA_Obj
import scipy.optimize as opt


#Inputs
fuel_type = 'C3H8:1'
oxidizer_type = 'O2:1'

pressure_fuel = 120; #psia
pressure_oxidizer = 120; #psia
pressure_ambient = 14.7; #psia
temp_fuel = 298.15; #K
temp_oxidizer = 298.15; #K


orifice_dia_fuel = 0.075; #in
orifice_dia_oxidizer = 0.106; #in
orifice_dia_outlet = 0.18; #in
cd = 0.6 #sharp-edged orifice assumption


#Define fluid properties from Cantera Library
fuel = ct.Solution('gri30.yaml')
oxidizer = ct.Solution('gri30.yaml')
fuel.TPX = temp_fuel, pressure_fuel*6894.76, fuel_type
oxidizer.TPX = temp_oxidizer, pressure_oxidizer*6894.76, oxidizer_type

rho_fuel = fuel.density #kg/m^3
rho_oxidizer = oxidizer.density #kg/m^3
gamma_fuel = fuel.cp/fuel.cv
gamma_oxidizer = oxidizer.cp/oxidizer.cv

print("Fuel Density [kg/m^3]:", rho_fuel)
print("Oxidizer Density [kg/m^3]:", rho_oxidizer)
print("Fuel Gamma:", gamma_fuel)
print("Oxidizer Gamma:", gamma_oxidizer)

#derived quantities
orifice_area_fuel = math.pi/4*(orifice_dia_fuel*0.0254)**2
orifice_area_oxidizer = math.pi/4*(orifice_dia_oxidizer*0.0254)**2
orifice_area_out = math.pi/4*(orifice_dia_outlet*0.0254)**2
pressure_fuel = pressure_fuel*6894.76 #Pa
pressure_oxidizer = pressure_oxidizer*6894.76 #Pa
pressure_ambient = pressure_ambient*6894.76 #Pa

#guess a chamber pressure
pressure_guess = 50 #psia
pressure_chamber = pressure_guess*6894.76 #Pa



print("Chamber Pressure: ", pressure_chamber/6894.76)
print()

def mass_balance(pressure_chamber):
    #use the critical pressure ratio to decide whether flow is choked or not
    pressure_critical_fuel = pressure_fuel*(2/(gamma_fuel+1))**(gamma_fuel/(gamma_fuel-1))
    pressure_critical_oxidizer = pressure_oxidizer*(2/(gamma_oxidizer+1))**(gamma_oxidizer/(gamma_oxidizer-1))

    if pressure_chamber <= pressure_critical_fuel:
        mdot_fuel = cd*orifice_area_fuel*math.sqrt(gamma_fuel*rho_fuel*pressure_fuel*(2/(gamma_fuel+1))**((gamma_fuel+1)/(gamma_fuel-1)))
    else:     
        mdot_fuel = cd*orifice_area_fuel*math.sqrt(2*rho_fuel*pressure_fuel*(gamma_fuel/(gamma_fuel-1))*((pressure_chamber/pressure_fuel)**(2/gamma_fuel)-(pressure_chamber/pressure_fuel)**((gamma_fuel+1)/gamma_fuel)))
    
    if pressure_chamber <= pressure_critical_oxidizer:
        mdot_oxidizer = cd*orifice_area_oxidizer*math.sqrt(gamma_oxidizer*rho_oxidizer*pressure_oxidizer*(2/(gamma_oxidizer+1))**((gamma_oxidizer+1)/(gamma_oxidizer-1)))
    else:
        mdot_oxidizer = cd*orifice_area_oxidizer*math.sqrt(2*rho_oxidizer*pressure_oxidizer*(gamma_oxidizer/(gamma_oxidizer-1))*((pressure_chamber/pressure_oxidizer)**(2/gamma_oxidizer)-(pressure_chamber/pressure_oxidizer)**((gamma_oxidizer+1)/gamma_oxidizer)))
   
    MR = mdot_oxidizer/mdot_fuel

    #bring in NASA CEA to get thermophysical properties of the combustion products
    cea = CEA_Obj(oxName = 'O2', fuelName = 'C3H8')
    Isp, cstar, temp_chamber, MW, gamma_chamber = cea.get_IvacCstrTc_ChmMwGam(pressure_chamber/6894.76, MR)
    rho_chamber = pressure_chamber*MW/(8314.5*temp_chamber) #kg/m^3
    print(mdot_fuel, mdot_oxidizer, MR, temp_chamber)
    pressure_critical_outlet = pressure_chamber*(2/(gamma_chamber+1))**(gamma_chamber/(gamma_chamber-1))

    if pressure_ambient < pressure_critical_outlet:
        mdot_out = cd*orifice_area_out*math.sqrt(gamma_chamber*rho_chamber*pressure_chamber*(2/(gamma_chamber+1))**((gamma_chamber+1)/(gamma_chamber-1)))
    else:
        mdot_out = cd*orifice_area_out*math.sqrt(2*pressure_chamber*rho_chamber*(gamma_chamber/(gamma_chamber-1))*((pressure_ambient/pressure_chamber)**(2/gamma_chamber)-(pressure_ambient/pressure_chamber)**((gamma_chamber+1)/gamma_chamber)))

    #Continuity Equation
    residue = mdot_fuel + mdot_oxidizer - mdot_out
    return residue

''''sol = opt.root_scalar(mass_balance, bracket=[pressure_ambient+1, min(pressure_fuel, pressure_oxidizer)-1], method='bisect')

if sol.converged:
    Pc_solution = sol.root
    print(f"Chamber Pressure: {Pc_solution/6894.76:.2f} psia")
else:
    print("Solver failed.")'''







Fuel Density [kg/m^3]: 14.717716915106001
Oxidizer Density [kg/m^3]: 10.679581510070117
Fuel Gamma: 1.127428664408443
Oxidizer Gamma: 1.3947290096372094
0.00378375949424106 0.006943875293127132 1.8351788224636945 4806.712538513277
9.278247711471708e-06 1.578745510347942e-05 1.7015556810322785 4617.91894295812
0.00378375949424106 0.006928104527854459 1.831010807742707 4930.784375152998
0.0033482070346787076 0.005880422060989383 1.7562898590450116 4763.709821724071
0.0025864859992809322 0.004466809433090242 1.726979938933386 4688.018729249362
0.0019035302461585784 0.003262353110791599 1.7138435900218527 4652.307415311435
0.0013719359040346673 0.002342717825492364 1.7076000552232549 4634.9510332429045
0.0009792148854849618 0.001669124375725309 1.7045537199923853 4626.395729299664
0.0011937131381871706 0.0020365590806132746 1.706070759769043 4630.66376651387
0.0012863444444675303 0.0021955762585457216 1.7068338639691167 4632.8050214836085
0.0012410045735712786 0.0021177146465365866 1.70645